In [1]:
import os
import yaml
import cv2
    
import numpy as np
from tqdm import tqdm
from ultralytics import YOLO
import shutil
import warnings
warnings.filterwarnings('ignore')

# Prepare data

In [15]:
model_names = ['yolov8n', 'yolov8s', 'yolov8m', 'yolov8l', 'yolov8x']
# model_names = ['yolov8x']
models_ncc = ['faster_rcnn', 'swin', 'retinanet']
f_data_path = '/home/user/2d/OUTPUTS_CVPR/{}_voc_test2'
# coco_val_label_path = '/mnt/2d/YOLO-attack/datasets/coco2017/val/labels/val2017'
# coco_train_label_path = '/mnt/2d/YOLO-attack/unet/labels/train2017'
val_datasets = ['attack']

In [43]:
for model_name in models_ncc:
    # for val_dataset in val_datasets:
    data_path = f_data_path.format(model_name)
    print(f"[INFO] Processing path: {data_path}")
    # imgs_base_path = os.path.join(data_path, 'base', "images", "val")
    # labels_base_path = os.path.join(data_path, 'base', "labels", "val")
    # os.makedirs(imgs_base_path, exist_ok=True)
    # os.makedirs(labels_base_path, exist_ok=True)

    imgs_attack_path = os.path.join(data_path, 'attack', 'images', 'val')
    # labels_attack_path = os.path.join(data_path, 'attack', 'labels', 'val')
    os.makedirs(imgs_attack_path, exist_ok=True)
    # os.makedirs(labels_attack_path, exist_ok=True)

    # Copy img base/attack vào folder mới
    imgs_path = os.path.join(data_path, 'images_full', 'val')
    for img_name in os.listdir(imgs_path):
        img_real_path = os.path.join(imgs_path, img_name)
        imgs = img_name.split('_')
        if imgs[-1] == 'base':  #base img
            continue
            # img_base_path = os.path.join(imgs_base_path, imgs[0] + '.jpg')
            # shutil.copy(img_real_path, img_base_path)
        else:               #attack img
            img_attack_path = os.path.join(imgs_attack_path, imgs[0] + "_" + imgs[1] + '.jpg')
            shutil.copy(img_real_path, img_attack_path)

    # Copy label vaf folder cua base/attack
    # shutil.copytree(coco_val_label_path, labels_base_path, dirs_exist_ok=True)
    # shutil.copytree(coco_val_label_path, labels_attack_path, dirs_exist_ok=True)
    print("Done!")
    print("-----------------------------------------------------------------------------------------------")

[INFO] Processing path: /home/user/2d/OUTPUTS_CVPR/voc_yolov8x.pt


Done!
-----------------------------------------------------------------------------------------------


# Inference với model.predict()

In [3]:
def yolo2coco(xyxy, img_width, img_height): # [15.69, 406.17, 344.22, 627.35], 480, 640
    x_min, y_min, x_max, y_max = xyxy
    w = x_max - x_min
    h = y_max - y_min
    
    x = (x_min + w/2) / img_width
    y = (y_min + h/2) / img_height

    w = w/img_width
    h = h/img_height
    return x, y, w, h

In [1]:
%cd datasets
%cd attacked
!ln -s /home/user/2d/OUTPUTS_CVPR/faster_rcnn_voc_0.98 faster_rcnn_voc_0.98
!ln -s /home/user/2d/OUTPUTS_CVPR/retina_voc_0.97 retina_voc_0.97
!ls

/home/user/2d/YOLOv8/datasets
/home/user/2d/YOLOv8/datasets/attacked


faster_rcnn_coco_0.98  pred_label	 retina_voc_0.97
faster_rcnn_voc_0.98   retina_coco_0.97  swin_coco_0.96


In [11]:
# val_datasets = ['attack']
['yolov8n', 'yolov8s', 'yolov8m', 'yolov8l', 'yolov8x']
models_ncc = ['swin']
# f_data_path = '/home/user/2d/OUTPUTS_CVPR/{}_voc_test'
img_path = "/home/user/2d/OUTPUTS_CVPR/{}_voc_test2"

In [12]:
for model_ncc in models_ncc:
    print(f"[INFO] Processing path: {model_ncc}")
    # img_attacked_path = os.path.join(img_path, "attack/images/val")
    img_attacked_path = img_path.format(model_ncc)
    for model_name in model_names:
        print(f"[INFO] Using {model_name}....")
        
        model = YOLO(f"/home/user/2d/YOLOv8/{model_name}.pt")
        model.conf = 0.5

        imgs_list = [os.path.join(img_attacked_path, img_name) for img_name in os.listdir(img_attacked_path)]

        for img in tqdm(imgs_list):
            result = model.predict(img)[0]
            h_img, w_img = result.orig_shape
            boxes = result.boxes.data.cpu().numpy()
            out = []
            for box in boxes:
                cls = int(box[-1])
                xyxy = box[:4]
                score = float(box[-2])
                
                x, y, w, h = yolo2coco(xyxy, w_img, h_img)
                
                out.append([cls, x, y, w, h, score])

            # Save x, y, w, h format coco to txt file
            img_name = os.path.split(result.path)[-1].split('.')[0]
            # pred_label_path = os.path.join(os.path.dirname(coco_dict['path']), model_name, img_type)
            pred_label_path = os.path.join(img_path, "pred_label", model_ncc, model_name)
            if not os.path.exists(pred_label_path):
                os.makedirs(pred_label_path, exist_ok=True)

            np.savetxt(os.path.join(pred_label_path, img_name + '.txt'), out, fmt='%0.4f')


    print("[DONE PATH]")
        

[INFO] Processing path: swin
[INFO] Using yolov8n....


  0%|          | 0/1804 [00:00<?, ?it/s]

100%|██████████| 1804/1804 [00:51<00:00, 35.28it/s]


[INFO] Using yolov8s....


100%|██████████| 1804/1804 [00:48<00:00, 37.47it/s]


...

[INFO] Using yolov8l....


100%|██████████| 1804/1804 [00:57<00:00, 31.29it/s]


[INFO] Using yolov8x....


100%|██████████| 1804/1804 [01:00<00:00, 29.59it/s]

[DONE PATH]


# Convert to json (evaluate with cocoapi)

In [13]:
import os
import json
import argparse
import cv2
import os
import math
from itertools import chain
import random
import numpy as np
from icecream import ic
import multiprocessing as mp
import shutil
from tqdm import tqdm
import imagesize

In [14]:
base_path = "/home/user/2d/OUTPUTS_CVPR/voc_test2/pred_label/{}"
img_path = "/home/user/2d/OUTPUTS_CVPR/{}_voc_test2"
base_json_path = "/home/user/2d/YOLOv8/datasets/coco2017/annotations/instances_val2017.json"
output_zip_path = "result_attacked/voc_new"
os.makedirs(output_zip_path, exist_ok=True)

coco_json = json.load(open(base_json_path))
category_name_dict_coco = {v["name"]: v["id"] for v in coco_json["categories"]}

category_ids_dict_yolo = {
  0: "person",
  1: "bicycle",
  2: "car",
  3: "motorcycle",
  4: "airplane",
  5: "bus",
  6: "train",
  7: "truck",
  8: "boat",
  9: "traffic light",
  10: "fire hydrant",
  11: "stop sign",
  12: "parking meter",
  13: "bench",
  14: "bird",
  15: "cat",
  16: "dog",
  17: "horse",
  18: "sheep",
  19: "cow",
  20: "elephant",
  21: "bear",
  22: "zebra",
  23: "giraffe",
  24: "backpack",
  25: "umbrella",
  26: "handbag",
  27: "tie",
  28: "suitcase",
  29: "frisbee",
  30: "skis",
  31: "snowboard",
  32: "sports ball",
  33: "kite",
  34: "baseball bat",
  35: "baseball glove",
  36: "skateboard",
  37: "surfboard",
  38: "tennis racket",
  39: "bottle",
  40: "wine glass",
  41: "cup",
  42: "fork",
  43: "knife",
  44: "spoon",
  45: "bowl",
  46: "banana",
  47: "apple",
  48: "sandwich",
  49: "orange",
  50: "broccoli",
  51: "carrot",
  52: "hot dog",
  53: "pizza",
  54: "donut",
  55: "cake",
  56: "chair",
  57: "couch",
  58: "potted plant",
  59: "bed",
  60: "dining table",
  61: "toilet",
  62: "tv",
  63: "laptop",
  64: "mouse",
  65: "remote",
  66: "keyboard",
  67: "cell phone",
  68: "microwave",
  69: "oven",
  70: "toaster",
  71: "sink",
  72: "refrigerator",
  73: "book",
  74: "clock",
  75: "vase",
  76: "scissors",
  77: "teddy bear",
  78: "hair drier",
  79: "toothbrush",
}

In [16]:
for model_ncc in models_ncc:
    base_path_ncc = base_path.format(model_ncc)
    print(f"[INFO] Processing path: {base_path_ncc}")
    for model_name in model_names:
        print(f"[INFO] Using {model_name}....")
        yolo_detect_txt_path = os.path.join(base_path_ncc, model_name)

        image_dir = img_path.format(model_ncc)
        
        detect_json = []
        total_annotation_count_pred = 0

        for idx, image_name in tqdm(enumerate(os.listdir(image_dir))):
            width, height = imagesize.get(os.path.join(image_dir, image_name))
            if (os.path.exists(os.path.join(yolo_detect_txt_path, image_name.replace("jpg", "txt"))) is False):
                continue
            
            with open(os.path.join(yolo_detect_txt_path, image_name.replace("jpg", "txt"))) as f:
                    annotations = f.readlines()

            for annotation in annotations:
                annotation = annotation.strip().split(" ")
                bbox = [float(i) for i in annotation[1:]]
                x1, y1, w, h = bbox[0], bbox[1], bbox[2], bbox[3]
                score = float(bbox[4])
                x1 = (x1 - w / 2) * width
                y1 = (y1 - h / 2) * height
                w = w * width
                h = h * height
                detect_json.append(
                    {
                        "image_id": os.path.splitext(image_name)[0],
                        "category_id": category_name_dict_coco[
                            category_ids_dict_yolo[int(float(annotation[0]))]
                        ],
                        "bbox": [x1, y1, w, h],
                        # "score": float(annotation[5]),
                        "score": score,
                    }
                )
                total_annotation_count_pred += 1
    

        os.makedirs(os.path.join(output_zip_path, model_ncc, model_name), exist_ok=True)
        with open(os.path.join(output_zip_path, model_ncc, model_name, "detect_attack.json"), "w") as f:
            json.dump(detect_json, f)
            
                
            

[INFO] Processing path: /home/user/2d/OUTPUTS_CVPR/voc_test2/pred_label/faster_rcnn
[INFO] Using yolov8n....


1215it [00:00, 16994.81it/s]


[INFO] Using yolov8s....


1215it [00:00, 22108.04it/s]


[INFO] Using yolov8m....


...

3595it [00:00, 15050.96it/s]


[INFO] Using yolov8l....


3595it [00:00, 10912.79it/s]


[INFO] Using yolov8x....


3595it [00:00, 15437.27it/s]


# Create new label for gt voc (instance_val2017.json)

In [17]:
import os
import argparse
import json
import shutil
import xml.etree.ElementTree as ET
from typing import Dict, List
from tqdm import tqdm
import re

In [18]:
voc_coco_path = "data/voc_coco"
# imgs_new_path = os.path.join(voc_coco_path, "val")
# os.makedirs(imgs_new_path, exist_ok=True)

In [19]:
def get_image_info(annotation_root, extract_num_from_imgid=False):
    path = annotation_root.findtext('path')
    if path is None:
        filename = annotation_root.findtext('filename')
    else:
        filename = os.path.basename(path)
    img_name = os.path.basename(filename)
    img_id = os.path.splitext(img_name)[0]
    if extract_num_from_imgid and isinstance(img_id, str):
        img_id = int(re.findall(r'\d+', img_id)[0])

    size = annotation_root.find('size')
    width = int(size.findtext('width'))
    height = int(size.findtext('height'))

    image_info = {
        'file_name': filename,
        'height': height,
        'width': width,
        'id': img_id,
        "license": 0,
        "flickr_url": "",
        "coco_url": "",
        "date_captured": 0,
    }
    return image_info

# Create new label for gt coco (instance_val2017.json)

In [22]:
coco_json_path = "/home/user/2d/mmdetection/data/coco/annotations/instances_val2017.json"
coco_json = json.load(open(base_json_path))
category_idx_dict_coco = {v["name"]: v["id"] for v in coco_json["categories"]}
voc_to_coco = {
    'aeroplane': 'airplane',
    'bicycle': 'bicycle',
    'bird': 'bird',
    'boat': 'boat',
    'bottle': 'bottle',
    'bus': 'bus',
    'car': 'car',
    'cat': 'cat',
    'chair': 'chair',
    'cow': 'cow',
    'diningtable': 'dining table',
    'dog': 'dog',
    'horse': 'horse',
    'motorbike': 'motorcycle',
    'person': 'person',
    'pottedplant': 'potted plant',
    'sheep': 'sheep',
    'sofa': 'couch',
    'train': 'train',
    'tvmonitor': 'tv'
}

voc_ids_dict = {
    0: 'aeroplane',
    1: 'bicycle',
    2: 'bird',
    3: 'boat',
    4: 'bottle',
    5: 'bus',
    6: 'car',
    7: 'cat',
    8: 'chair',
    9: 'cow',
    10: 'diningtable',
    11: 'dog',
    12: 'horse',
    13: 'motorbike',
    14: 'person',
    15: 'pottedplant',
    16: 'sheep',
    17: 'sofa',
    18: 'train',
    19: 'tvmonitor',
}

In [50]:
len(os.listdir("/home/user/2d/OUTPUTS_CVPR/voc_yolov8x.pt/attack/images/val"))

3701

In [26]:
image_dir = "/home/user/2d/OUTPUTS_CVPR/swin_voc_test2"
f = open('/home/user/2d/mmdetection/data/VOCdevkit/VOC2012/ImageSets/Main/new_val_swin_voc_test2.txt', 'w')
for img_name in sorted(os.listdir(image_dir)):
    f.write(os.path.splitext(img_name)[0] + "\n")

In [27]:
yolo_txt_path = "/home/user/2d/datasets/VOC/labels/val2012"
image_dir = "/home/user/2d/OUTPUTS_CVPR/swin_voc_test2"


images_info = []
annotations_coco = []
total_annotation_count = 1
total_annotation_count_gt = 0
total_anno_file = 0

for idx, image_name in tqdm(enumerate(os.listdir(image_dir))):
    width, height = imagesize.get(os.path.join(image_dir, image_name))
    images_info.append(
        {
            "id": os.path.splitext(image_name)[0],
            "file_name": image_name,
            "width": width,
            "height": height,
            "license": 0,
            "flickr_url": "",
            "coco_url": "",
            "date_captured": 0,
        }
    )

    if (os.path.exists(os.path.join(yolo_txt_path, image_name.replace("jpg", "txt"))) is False):
            continue
    
    with open(os.path.join(yolo_txt_path, image_name.replace("jpg", "txt"))) as f:
        annotations = f.readlines()
        total_anno_file += 1
    
    for annotation in annotations:
        # convert yolo bounding boxes to coco
        annotation = annotation.strip().split(" ")
        
        category_id_coco = category_idx_dict_coco[voc_to_coco[voc_ids_dict[int(annotation[0])]]]

        bbox = [float(i) for i in annotation[1:5]]
        x1, y1, w, h = bbox[0], bbox[1], bbox[2], bbox[3]
        x1 = (x1 - w / 2) * width
        y1 = (y1 - h / 2) * height
        w = w * width
        h = h * height
        annotations_coco.append(
            {
                "id": total_annotation_count,
                "image_id": os.path.splitext(image_name)[0],
                "category_id": category_id_coco,
                "area": w * h,
                "segmentation": [],
                "bbox": [x1, y1, w, h],
                "iscrowd": 0,
                "attributes": {"occluded": False},
            }
        )
        total_annotation_count += 1
        total_annotation_count_gt += 1
coco_json["images"] = images_info
coco_json["annotations"] = annotations_coco

print(total_annotation_count_gt)
print(total_anno_file)
# os.makedirs(os.path.join(output_zip_path), exist_ok=True)
with open(os.path.join(output_zip_path, "instances_defaults_swin_voc_test2.json"), "w") as f:
    json.dump(coco_json, f)

1804it [00:00, 14300.12it/s]

4130
1804


# Result

In [42]:
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

annType = "bbox"

cocoGt = COCO("/home/user/2d/YOLOv8/result_attacked/voc_new/instances_defaults_swin_voc_test2.json")
cocoDt = cocoGt.loadRes("/home/user/2d/YOLOv8/result_attacked/voc_new/swin/yolov8m/detect_attack.json")

imgIds = sorted(cocoGt.getImgIds())
print(len(imgIds))

# running evaluation
cocoEval = COCOeval(cocoGt, cocoDt, annType)
cocoEval.params.imgIds = imgIds
# cocoEval.params.catIds = [3]
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
1804
Running per image evaluation...
Evaluate annotation type *bbox*


DONE (t=0.62s).
Accumulating evaluation results...
DONE (t=0.13s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.1208
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.1705
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.1275
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.0320
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.0509
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.1453
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.1206
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.1347
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.1348
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.0329
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.0602
 Average Recall     (AR) @[ IoU=0.50:0.95 